# Séance 1 - EarlyDropout

On se propose dans ce notebook de reproduire une partie de l'article [Dropout reduces underfitting](https://arxiv.org/abs/2303.01500). Pour le vérifier, nous allons utiliser le dataset Fashion MNIST. Commençons par importer les packages dont nous aurons besoin, importer et standardiser les données.

In [ ]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_valid, y_valid) = (fashion_mnist.load_data())


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)
X_valid = scaler.transform(X_valid.astype(np.float32).reshape(-1, 28 * 28)).reshape(-1, 28, 28, 1)


label_map = {0: "t-shirt/top", 1: "trouser", 2: "pullover",
             3: "dress", 4: "coat", 5: "sandal",
             6: "shirt", 7: "sneaker", 8: "bag", 9: "ankle boot"}

Visualisons quelques exemples :

In [ ]:
n = 5

plt.figure(figsize=(13, 6))
for plot_index in range(1, n+1):
  plt.subplot(1, n, plot_index)
  index = np.random.randint(low=0, high=len(X_train))
  plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
  plt.title("Image of a %s" % label_map[y_train[index]])
  plt.axis('off')
plt.show()

## EarlyDropout

L'early dropout tel que définit dans l'article correspond à avoir la probabilité de dropout strictement supérieure à zéro jusqu'à une certaine époque. Après la valeur vaut 0.

Pour pouvoir reproduire ce fonctionnement, nous allons devoir définir notre propre [callback](https://keras.io/guides/writing_your_own_callbacks/) pour modifier le réseau de neurones pendant l'entraînement.
Nous aurons besoin de :
* La valeur initiale du taux de dropout
* L'époque à laquelle on devra *arrêter* le dropout

**Consigne** : Ecrire la classe *EarlyDropout* selon le template suivant. Pour information, le paramètre *epoch* vaut 0 pour l'époque numéroté 1 dans l'affichage habituel.


In [ ]:
class EarlyDropout(keras.callbacks.Callback):
  def __init__(self, ...):
    ...

  def on_epoch_end(self, epoch, logs=None):
    ...

## Réseau de neurones
Définissons à présent le réseau convolutionnel que l'on va exploiter. Pour pouvoir simplement mesurer l'apport (ou non) de l'early dropout, nous allons définir une fonction qui définira, entraînera et renverra les performances de l'entraînement du réseau de neurones.
Nous aurons besoin de définir les paramètres:
* **Early dropout** : la valeur du dropout et l'époque de fin d'utilisation
* **Entraînement** : le learning rate et le nombre d'époque

**Consigne** : Renseigner la fonction *generate_train_model* suivante. On ajoutera un paramètre *verbose* à la fonction pour contrôler le même paramètre dans l'appel *.fit* pour ne pas avoir la totalité des affichages.
On définira le variable *dropout_rate_variable* avec une valeur réelle à spécifier à l'aide de la fonction *keras.backend.variable*. De cette manière on pourra la modifier avec la classe EarlyDropout précédente.
Vous êtes libre de modifier à volonté la définition du réseau de neurones.

In [ ]:
def generate_train_model(...):
  dropout_rate_variable = keras.backend.variable(...)

  model = keras.models.Sequential([
    ...
  ])

  callback = EarlyDropout(...)

  model.compile(...)
  history = model.fit(...)
  return pd.DataFrame(history.history)

## Comparer plusieurs choix

Pour simplifier l'écriture de multiple test, et simplifier le stockage des historiques d'apprentissage, on se propose d'utiliser une fonction pour le faire.
Elle nécessite une liste d'expériences à réaliser. Chaque expérience est un dictionnaire que l'on paramètre avec:
* **type** : le nom de l'expérience
* **function** : la fonction qui génére l'historique d'apprentissage
* **parameters** : les paramètres à renseigner pour appeler la fonction *function*

Cette fonction réalisera l'ensemble des expériences un certain nombre de fois pour pouvoir potentiellement estimer un interval de confiance.

In [ ]:
def compare_architectures(comparisons, comparisons_number=2):
  results = []


  for index in range(comparisons_number):
    print("Comparison %d :" % (index+1), end=" ")

    for element in comparisons:
      print("%s..." % element["type"], end= " ")
      function, parameters = element["function"], element["parameters"]
      history = function(**parameters)
      result = {"type": element["type"], "history": history}
      results.append(result)

    print()


  return results

Il ne nous reste plus qu'à générer le dictionnaire d'expériences ! Nous entraînerons sur 20 époques uniquement les réseaux de neurones. Nous considérons ici uniquement trois expériences:
* Un réseau avec un EarlyDropout au bout de 10 époques
* Un réseau avec un EarlyDropout au bout de 20 époques, autrement dit avec dropout tout au long de l'entraînement
* Un réseau avec un EarlyDropout au bout de 0 époques, autrement dit sans dropout tout au long de l'entraînement

**Consigne** : Remplir la cellule ci-dessous pour générer les expériences que l'on souhaite réaliser en accord avec les informations précédentes.

In [ ]:
dropout_rate = 0.5
learning_rate = 5e-4
n_epochs = ...
epochs = [...]


comparisons = []

for epoch in epochs:
  parameters = {...}

  comparison = {"type": "EarlyDropout %d epochs" % epoch,
                "function": generate_train_model,
                "parameters": parameters}

  comparisons.append(comparison)

**Consigne** : Appeler la fonction précédente pour lancer les comparaisons définies à la cellule précédente.

In [ ]:
...

Nous venons de générer un objet *results* qui contient les types et les historiques des différentes expériences réalisées. Nous souhaitons les visualiser de manière simple.

## Visualisation

On se propose de construire une fonction qui exploite spécifiquement cette architecture d'objet pour le faire. Nous générerons deux figures:
1. La valeur de la fonction de perte en fonction du temps d'entraînement
2. La valeur de l'accuracy en fonction du temps d'entraînement

Dans ces deux graphiques, nous présenterons en trait pleins la valeur de l'entraînement et en trait pointillés la valeur de la validation. Nous présenterons chaque type d'expérience sur chaque graphique accompagné de l'intervalle de confiance pour l'entraînement comme pour la validation.

In [ ]:
def plot_comparison(results,  title, confidence=3, figsize=(15, 6), alpha=0.1):

    def agregate_result(key, metric_name):
        training = np.zeros((comparisons_number, n_epochs))
        validation = np.zeros((comparisons_number, n_epochs))
        index = 0
        for result in results:
            if result["type"] == key:
                historic = result["history"]
                training[index] = historic[metric_name]
                validation[index] = historic["val_%s" % metric_name]
                index += 1
        return training, validation


    def get_vector_mean_std(vector):
        mean = vector.mean(axis=0)
        std = vector.std(axis=0)
        return mean, std



    n_epochs = results[0]["history"].shape[0]
    epochs = range(1, n_epochs+1)
    comparisons_number = sum([result["type"] == results[0]["type"] for result in results])
    figure, (axis_1, axis_2) = plt.subplots(1, 2, figsize=figsize)
    keys = list(set([result["type"] for result in results]))


    for metric_name, axis in zip(["loss", "accuracy"], [axis_1, axis_2]):

        for index, key in enumerate(keys):
            color = sns.color_palette()[index]
            training, validation = agregate_result(key, metric_name)

            training_mean, training_std = get_vector_mean_std(training)
            axis.plot(epochs, training_mean, lw=2, label=key, color=color)
            axis.fill_between(epochs, training_mean-confidence*training_std, training_mean+confidence*training_std, color=color, alpha=alpha)

            validation_mean, validation_std = get_vector_mean_std(validation)
            axis.plot(epochs, validation_mean, ls="--", color=color)
            axis.fill_between(epochs, validation_mean-confidence*validation_std, validation_mean+confidence*validation_std, color=color, alpha=alpha)

        axis.set_ylabel(metric_name.capitalize())
        axis.set_xlabel("Epochs")
        axis.set_title("%s through training" % metric_name.capitalize())
        axis.legend()


    plt.suptitle(title)
    plt.show()

**Consigne** : utiliser la fonction précédente pour visualiser les courbes d'apprentissages.

In [ ]:
...

**Consigne** : Commenter le précédent graphique.